일단 시피유로 인퍼런스 ㄱㄱ
1. 모델 불러오기 (나중에는 항상 메모리에 올려놓기 ㄱㄱ)
2. 데이터 단일 혹은 배치단위로 읽어오기
    - 일단 npy
    - 나중에 xml을 받아야함..

In [1]:
### json style ###

CFG = {
    'DEVICE' : 'cpu',
    'TARGET_COLS' : [
        'ECG_event_3d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_3d_CIA-keyword_v2_onset',
        'ECG_event_7d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_7d_CIA-keyword_v2_onset',
        'ECG_event_14d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_14d_CIA-keyword_v2_onset',
        'ECG_event_30d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_30d_CIA-keyword_v2_onset',
        'ECG_event_90d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_90d_CIA-keyword_v2_onset',
        'AFIB_AFL-keyword_v2',
        'CIA-keyword_v2'
    ],
    'MODEL_INFO' : {
        'config':{
            'embed_dim':768,  
            'patch_size':32, 
            'seq_length':2560,
            'in_channels':12, 
            'encoder':'vit_encoder',
            'merge_mode':'projection',  
            'num_classes':12,
        },        
        'model_path_list': [
            '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_AUPRC-Ep_3-Lo_0.082-M0_0.793-M1_0.494.pth',
            '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_AUROC-Ep_7-Lo_0.082-M0_0.800-M1_0.493.pth',
            '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_loss-Ep_5-Lo_0.082-M0_0.797-M1_0.493.pth',
        ],
    }
}


In [2]:
from model_architecture import SynAI
import torch

def model_build(model_info):
    model = SynAI.MAE_1D_250409_v3(
        seq_length  = model_info['seq_length'],
        in_channels = model_info['in_channels'],
        patch_size  = model_info['patch_size'],
        embed_dim   = model_info['embed_dim'],
        merge_mode  = model_info['merge_mode'],  # linear_projection avg add
        encoder     = model_info['encoder'],
    )
    model = SynAI.OnlyEncoderForFT_250409(
        model,
        num_classes = model_info['num_classes'],
        embed_dim = model_info['embed_dim'],
    )
    return model

def load_models(model_path_list, model_info, device='cpu'):
    models = []
    for model_path in model_path_list:
        checkpoint = torch.load(model_path, weights_only=True)
        model = model_build(model_info).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])
        models.append({'model_path': model_path, 'model': model})
    return models



In [3]:
models = load_models(CFG['MODEL_INFO']['model_path_list'], 
                     CFG['MODEL_INFO']['config'], device=CFG['DEVICE'])


/home/syai-research/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


seq_length 2560, in_channels 12, patch_size 32, embed_dim 768, token_len 80, 
seq_length 2560, in_channels 12, patch_size 32, embed_dim 768, token_len 80, 
seq_length 2560, in_channels 12, patch_size 32, embed_dim 768, token_len 80, 


In [4]:
from torch.utils.data import DataLoader
from processing import transforms, dataset

transform_pipeline = {}
transform_pipeline['val'] = transforms.TransformPipeline([
    transforms.NormalizeECG(method="tanh", scope="lead-wise", scale=1),
])


In [19]:
import pandas as pd
df_path = '/mnt/home/Datasets_processed/EKG_Latest/processed_metadata_v6-sv2_subset_2_holdout.csv'
df = pd.read_csv(df_path)
tmp_npy_path = df['bpf0540_or3-npy_path'][:2]
tmp_npy_path.to_list()

/tmp/ipykernel_1563535/1969612414.py:3: DtypeWarning: Columns (4,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path)


['/home/Datasets_processed/EKG_Latest/signal-bpf0540_or3/10302870_2023-10-17_2023101716092374_2023101716095795.npy',
 '/home/Datasets_processed/EKG_Latest/signal-bpf0540_or3/10478425_2024-02-28_2024022823332301_2024022823344711.npy']

In [20]:
bandpass_info = {
    "lowcut": 0.5,
    "highcut": 40.0,
    "order": 3,
}

In [21]:
import numpy as np
import torch
from processing import transforms, dataset

# 배치 데이터를 저장할 리스트
batch_ecg = []

for i in range(len(tmp_npy_path)):
    # ECG 데이터 로드 및 전처리
    tmp_ecg = np.load('/mnt' + tmp_npy_path[i])
    tmp_ecg = dataset.ECGDataset_v2.preprocess_ecg(
        x               = tmp_ecg, 
        scaling_factor  = None,
        do_bandpass     = bandpass_info,
        original_fs     = 500,
        target_fs       = 256,
        do_normalize    = False,
        transforms      = transform_pipeline['val']
    )
    # 배치에 추가
    batch_ecg.append(tmp_ecg)

# 리스트를 PyTorch 텐서로 변환 (배치 크기, 채널, 시퀀스 길이)
batch_ecg_tensor = torch.tensor(np.stack(batch_ecg), dtype=torch.float32)

# 결과 확인
print("배치 텐서 크기:", batch_ecg_tensor.shape)  # 예: (배치 크기, 채널, 시퀀스 길이)

배치 텐서 크기: torch.Size([2, 12, 2560])


In [26]:
import torch
from typing import Dict, Optional, List
from torch.utils.data import DataLoader
from tqdm import tqdm

@torch.no_grad()
def inference_tensor(
    model: torch.nn.Module,
    input_tensor: torch.Tensor,
    activation: Optional[str] = 'sigmoid',
    to_cpu: bool = True
) -> Dict[str, torch.Tensor]:
    """
    단일 샘플 또는 배치 텐서에 대해 추론 결과 반환
    """
    original_input = input_tensor.clone().detach()
    device = next(model.parameters()).device
    model.eval()
    inference_input = input_tensor.to(device)
    is_batch = inference_input.dim() >= 3
    if not is_batch:
        inference_input = inference_input.unsqueeze(0)
    with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
        logits = model(inference_input)
    if not is_batch:
        logits = logits.squeeze(0)
    probs = None
    if activation == 'sigmoid':
        probs = torch.sigmoid(logits)
    elif activation == 'softmax':
        probs = torch.softmax(logits, dim=-1)
    if to_cpu:
        logits = logits.cpu()
        if probs is not None:
            probs = probs.cpu()
    result = {
        "input": original_input,
        "logits": logits
    }
    if probs is not None:
        result["probs"] = probs
    return result


In [27]:
result = inference_tensor(
    models[0]['model'], batch_ecg_tensor)
result

{'input': tensor([[[-2.9693e-03, -7.0048e-03, -8.8557e-03,  ...,  1.9844e-03,
            1.3033e-03,  1.9885e-03],
          [-2.0600e-03, -4.3306e-03, -4.3705e-03,  ...,  3.5940e-03,
            2.6778e-03,  2.8508e-03],
          [ 1.7023e-03,  5.2238e-03,  8.3791e-03,  ...,  1.5170e-03,
            1.3776e-03,  1.0135e-03],
          ...,
          [-8.2133e-03, -1.0171e-02, -1.4388e-02,  ..., -1.4268e-02,
           -1.3296e-02, -1.3271e-02],
          [ 4.5172e-04,  1.5961e-02,  2.3923e-02,  ..., -1.3827e-02,
           -1.2010e-02, -1.2967e-02],
          [ 6.6286e-03,  3.4782e-02,  5.3068e-02,  ..., -7.7860e-03,
           -6.4454e-03, -8.2739e-03]],
 
         [[-3.4640e-03,  3.5442e-02,  7.9031e-02,  ..., -1.8068e-03,
           -1.8399e-03, -1.6192e-03],
          [ 2.9238e-02,  6.0985e-02,  7.3498e-02,  ..., -2.2764e-03,
            4.3108e-04, -3.7820e-03],
          [ 3.2890e-02,  2.5221e-02, -6.6650e-03,  ..., -9.2264e-06,
            2.5665e-03, -1.9017e-03],
          

In [28]:
result['probs'].shape

torch.Size([2, 12])

In [31]:
af_idx = CFG['TARGET_COLS'].index('ECG_event_14d_AFIB_AFL-keyword_v2_onset')
cia_idx = CFG['TARGET_COLS'].index('ECG_event_14d_CIA-keyword_v2_onset')

In [33]:
result['probs'][:, af_idx]
result['probs'][:, cia_idx]

tensor([0.0720, 0.1993])

In [37]:
import joblib
af_cal_model = joblib.load('/mnt/home/bgk/macai-engine-v1/trained_models/v1.2.1/calibration_model-AF.pkl')
af_prob_cal = af_cal_model.predict_proba(result['probs'][:, af_idx].reshape(-1, 1))[:, 1]
cia_cal_model = joblib.load('/mnt/home/bgk/macai-engine-v1/trained_models/v1.2.1/calibration_model-CIA.pkl')
cia_prob_cal = cia_cal_model.predict_proba(result['probs'][:, cia_idx].reshape(-1, 1))[:, 1]

In [38]:
af_high_g_thr = 0.18594001505695362
af_inter_g_thr = 0.09283757956391994
cia_high_g_thr = 0.6134156782179706
cia_inter_g_thr = 0.39689483565475614

In [42]:
def classify_risk(
    probs:np.array, high_thr:float, inter_thr:float, method:str=None):
    '''
        probs: 위험도 확률 리스트
        high_thr: 고위험군 기준 임계값 (상위 10%)
        inter_thr: 중간위험군 기준 임계값 (상위 10% ~ 상위 30%)
    '''
    results = []    
    if method == 'mean':
        probs = probs.mean()
        if probs >= high_thr:
            return [probs, "High"]
        elif probs >= inter_thr:
            return [probs, "Intermediate"]
        else:
            return [probs, "Low"]
    else:
        for prob in probs:
            if prob >= high_thr:
                results.append([prob, "High"])
            elif prob >= inter_thr:
                results.append([prob, "Intermediate"])
            else:
                results.append([prob, "Low"])
        return results

In [44]:
af_risk = classify_risk(af_prob_cal, af_high_g_thr, af_inter_g_thr, method='mean')
cia_risk = classify_risk(cia_prob_cal, cia_high_g_thr, cia_inter_g_thr, method='mean')
af_risk, cia_risk

([0.08155256296124591, 'Low'], [0.5112336937366079, 'Intermediate'])

## yaml test

In [23]:
import yaml

config_path = '/mnt/home/bgk/macai-engine-v1/trained_models/v1.2.1/config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

print(config)

{'MODEL_INFO': {'DEVICE': 'cpu', 'CONFIG': {'embed_dim': 768, 'patch_size': 32, 'seq_length': 2560, 'in_channels': 12, 'encoder': 'vit_encoder', 'merge_mode': 'projection', 'num_classes': 12}, 'MODEL_PATHS': ['/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_AUPRC-Ep_3-Lo_0.082-M0_0.793-M1_0.494.pth', '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_AUROC-Ep_7-Lo_0.082-M0_0.800-M1_0.493.pth', '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_loss-Ep_5-Lo_0.082-M0_0.797-M1_0.493.pth'], 'AF_CAL_MODEL_PATH': '/mnt/home/bgk/macai-engine-v1/trained_models/v1.2.1/calibration_model-AF.pkl', 'CIA_CAL_MODEL_PATH': '/mnt/home/bgk/macai-engine-v1/trained_models/v1.2.1/calibration_model-CIA.pkl', 'RISK_THRESHOLD': {'af_high_g_thr': 0.18594001505695362, 'af_inter_g_th

In [24]:
from processing import infer
models = infer.load_models(config['MODEL_INFO']['MODEL_PATHS'], 
                           config['MODEL_INFO']['CONFIG'], device=config['MODEL_INFO']['DEVICE'])

/home/syai-research/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


seq_length 2560, in_channels 12, patch_size 32, embed_dim 768, token_len 80, 
seq_length 2560, in_channels 12, patch_size 32, embed_dim 768, token_len 80, 
seq_length 2560, in_channels 12, patch_size 32, embed_dim 768, token_len 80, 


In [25]:
import joblib
af_cal_model = joblib.load(config['MODEL_INFO']['AF_CAL_MODEL_PATH'])
cia_cal_model = joblib.load(config['MODEL_INFO']['CIA_CAL_MODEL_PATH'])

In [27]:
from torch.utils.data import DataLoader
from processing import transforms, dataset, infer

transform_pipeline = {}
transform_pipeline['val'] = transforms.TransformPipeline([
    transforms.NormalizeECG(method="tanh", scope="lead-wise", scale=1),
])


### 임시 ###
import numpy as np
import torch

batch_ecg = []
for tmp_npy_path in config['DATA_INFO']['ECG_PATHS']:
    tmp_ecg = np.load(tmp_npy_path)
    tmp_ecg = dataset.ECGDataset_v2.preprocess_ecg(
        x               = tmp_ecg, 
        scaling_factor  = config['DATA_INFO']['PREPROCESS']['scaling_factor'],
        do_bandpass     = config['DATA_INFO']['PREPROCESS']['do_bandpass'],
        original_fs     = config['DATA_INFO']['PREPROCESS']['original_fs'],
        target_fs       = config['DATA_INFO']['PREPROCESS']['target_fs'],
        do_normalize    = config['DATA_INFO']['PREPROCESS']['do_normalize'],
        transforms      = transform_pipeline['val']
    )
    batch_ecg.append(tmp_ecg)
batch_ecg_tensor = torch.tensor(np.stack(batch_ecg), dtype=torch.float32)
print(batch_ecg_tensor.shape)

torch.Size([2, 12, 2560])


In [16]:
### 임시 ###
results = []
for i in range(len(models)):
    result = infer.inference_tensor(models[i]['model'], batch_ecg_tensor)
    results.append(result)
probs_list = [result['probs'] for result in results]  # results는 각 모델의 결과 리스트
stacked_probs = torch.stack(probs_list, dim=0)  # (모델 개수, 배치 크기, 클래스 개수)
mean_probs = stacked_probs.mean(dim=0)  # 모델 차원(0)을 기준으로 평균 계산
mean_probs = np.array(mean_probs)
print("평균 확률값:", mean_probs.shape)

평균 확률값: (2, 12)


In [18]:
af_idx = config['DATA_INFO']['OUTPUT_CLS_INFO']['cls_names'].index(config['DATA_INFO']['OUTPUT_CLS_INFO']['target_cls']['AF'])
cia_idx = config['DATA_INFO']['OUTPUT_CLS_INFO']['cls_names'].index(config['DATA_INFO']['OUTPUT_CLS_INFO']['target_cls']['CIA'])
af_prob_cal = af_cal_model.predict_proba(mean_probs[:, af_idx].reshape(-1, 1))[:, 1]
cia_prob_cal = cia_cal_model.predict_proba(mean_probs[:, cia_idx].reshape(-1, 1))[:, 1]

In [7]:
af_risk = infer.classify_risk(
    probs = af_prob_cal, 
    high_thr = config['MODEL_INFO']['RISK_THRESHOLD']['af_high_g_thr'], 
    inter_thr = config['MODEL_INFO']['RISK_THRESHOLD']['af_inter_g_thr'], 
    method = 'mean'
)
cia_risk = infer.classify_risk(
    probs = cia_prob_cal, 
    high_thr = config['MODEL_INFO']['RISK_THRESHOLD']['cia_high_g_thr'], 
    inter_thr = config['MODEL_INFO']['RISK_THRESHOLD']['cia_inter_g_thr'], 
    method = 'mean'
)
af_risk, cia_risk

([0.09766305002407535, 'Intermediate'], [0.5498921614999946, 'Intermediate'])